In [4]:
from dipy.io.streamline import load_tractogram
from dipy.io.stateful_tractogram import Space
from pathlib import Path

# Load the tractogram file (e.g., 'tractogram.trk')
tractogram = load_tractogram('./Stack_Synthetic_Data_Anti-Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen.trk',reference='same')

# Convert the streamlines to a numpy array in the RAS+ space
streamlines = tractogram.streamlines

# Extract the affine transformation matrix
affine = tractogram.affine

# Print the affine and the first streamline's coordinates as examples
print("Affine transformation matrix:")
print(affine)

# To get the coordinates of each streamline
streamline_coords = [s for s in streamlines]

# Print the number of streamlines
print("\nNumber of streamlines:")
print(len(streamline_coords))

FileNotFoundError: [Errno 2] No such file or directory: './Stack_Synthetic_Data_Anti-Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen.trk'

The following cells convert the tractogram into a [neuroglancer precomputed skeleton format](https://github.com/google/neuroglancer/blob/master/src/datasource/precomputed/skeletons.md).

The first version only supports unsharded format.

In [ ]:
ng_skeleton_path = Path("Stack_Synthetic_Data_Anti-Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen_ngskel")
ng_skeleton_path.mkdir(exist_ok=True)

In [ ]:
import json
import struct
import numpy as np

def write_info_file(dir_path, affine, sharded = False):
    affine_list_4x3 = affine[:3].flatten().tolist()

    if sharded:
        raise Exception("Not yet implemented")
    else:
        info = {
            "@type": "neuroglancer_skeletons",
            "transform": affine_list_4x3,
            "vertex_attributes": [
                # {
                #     "id": "radius",
                #     "data_type": "float32",
                #     "num_components": 1
                # }
            ],
            #"sharding":
            #"segment_properties": 
        }
    
    with open(dir_path / 'info', 'w') as file:
        json.dump(info, file, indent=4)  # `indent=4` makes the file readable with pretty print

def write_unsharded_skeleton_data(dir_path, streamline_coords):
    for i in range(len(streamline_coords)):
        vertex_positions = np.array(streamline_coords[i], dtype=np.float32)
        num_vertices = len(vertex_positions)
        num_edges = num_vertices - 1
        assert vertex_positions.shape == (num_vertices, 3)
        edges = np.array([(i, i + 1) for i in range(num_vertices - 1)], dtype=np.uint32)
        assert edges[-1][1] == num_vertices - 1
        assert edges.shape == (num_edges, 2)

        with open(dir_path / str(i), 'wb') as file:
            """"""
            file.write(struct.pack('<I', num_vertices))
            file.write(struct.pack('<I', num_edges))
            file.write(vertex_positions.astype('<f4').tobytes())
            file.write(edges.astype('<u4').tobytes())

write_info_file(ng_skeleton_path, affine, sharded=False)
write_unsharded_skeleton_data(ng_skeleton_path, streamline_coords)

The dataset is now converted. To be able to visualise it locally, you will need: 

1. a local neuroglancer server and 

2. a local HTTP server (with CORS resolver) to the dataset.

In [8]:
## 1. local neuroglancer server

import neuroglancer
import numpy as np

viewer = neuroglancer.Viewer()
print(viewer)

http://127.0.0.1:60470/v/292cb8269e5697b1f8f1be74334362348b4267c6/


In [9]:
## 2. local HTTP server (with CORS resolver) to the dataset

import http.server
import socketserver
import os

PORT = 8000
DIRECTORY = "."

class CORSRequestHandler(http.server.SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET')
        self.send_header('Access-Control-Allow-Headers', 'Content-Type')
        super().end_headers()

    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=DIRECTORY, **kwargs)

with socketserver.TCPServer(("", PORT), CORSRequestHandler) as httpd:
    print(f"Serving HTTP on port {PORT}...")
    httpd.serve_forever()


Serving HTTP on port 8000...


127.0.0.1 - - [08/Aug/2024 10:15:36] code 404, message File not found
127.0.0.1 - - [08/Aug/2024 10:15:36] "GET /info HTTP/1.1" 404 -
127.0.0.1 - - [08/Aug/2024 10:15:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:15:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:15:58] "GET /Stack_Synthetic_Data_Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen_ngskel/ HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:16:00] "GET /Stack_Synthetic_Data_Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen_ngskel/info HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:17:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:17:05] "GET /Stack_Synthetic_Data_Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen_ngsegprops/ HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:17:06] "GET /Stack_Synthetic_Data_Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen_ngsegprops/info HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:18:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2024 10:18:28] "GET 

In [7]:
import pandas as pd
from nglui.segmentprops import SegmentProperties
from pathlib import Path
import json

df = pd.DataFrame(
    {'segment_id': list(range(0,12520)),
     'fiber_type': ['cortical']*12520}
)

seg_props = SegmentProperties.from_dataframe(
    df,
    id_col='segment_id',
    tag_value_cols='fiber_type'
)

ng_segprops_path = Path("Stack_Synthetic_Data_Parallel_Cylinders_Random_Dia_512X512_filtered50vxlen_ngsegprops")
ng_segprops_path.mkdir(exist_ok=True)

with open(ng_segprops_path / 'info', 'w') as file:
    json.dump(seg_props.to_dict(), file, indent=4)  # `indent=4` makes the file readable with pretty print